In [15]:
#Implementation of a Multi Layer Perceptrónidas
#Getting more context than the last implementation. 

In [16]:
import torch
import torch.nn.functional as F
import matplotlib as plt 
%matplotlib inline

In [17]:
#read words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [18]:
len(words)

32033

In [19]:
#build vocabulary of characters and mappins to integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [20]:
#build the dataset

block_size = 3 #Context lenght (numbers of characters in this case)
X, Y = [], []
for w in words[:5]:

    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '------->', itos[ix]) #I dont understand this line of code at all
        context = context[1:] + [ix] #crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... -------> e
..e -------> m
.em -------> m
emm -------> a
mma -------> .
olivia
... -------> o
..o -------> l
.ol -------> i
oli -------> v
liv -------> i
ivi -------> a
via -------> .
ava
... -------> a
..a -------> v
.av -------> a
ava -------> .
isabella
... -------> i
..i -------> s
.is -------> a
isa -------> b
sab -------> e
abe -------> l
bel -------> l
ell -------> a
lla -------> .
sophia
... -------> s
..s -------> o
.so -------> p
sop -------> h
oph -------> i
phi -------> a
hia -------> .


In [24]:
C = torch.randn((27,2))
C[5]

tensor([1.2126, 0.0022])

tensor([[ 1.2126,  0.0022],
        [-1.2063,  1.4282],
        [ 1.5302, -0.5529]])